## Cubic Spline Interpolation

*Material for this worksheet can be found in the AE2220-I lecture notes Chapter 4.1.*

### Problem statement

In the previous module we saw that polynomial interpolation can be oscillatory. In this circumstance, higher degree polynomials, and more samples do not necessarily mean lower error (see notes, pg. 31). Furthermore, we saw this is especially the case with uniformly-spaced samples - and sometimes we are not free to choose $x_i$.  This is the case in e.g. most forms of time-sampling.

Once again we are concerned with approximating a smooth function of one variable $f(x)$, on the interval $[a, b]$:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
Ia, Ib = -1.0, 1.0                 ### Interval endpoints
N = 4                              ### Number of sub-intervals
xi = np.linspace(Ia,Ib,N+1)        ### Uniform sample locs  

### Functions to test - and 2nd derivatives

def f_sin(x): return np.sin(3*x)         ### Sine wave  
def d2fdx2_sin(x): return -3**2*np.sin(3*x)

def f_runge(x): return 1./(1+(4.*x)**2)  ### Runge fn
def d2fdx2_runge(x): 
    fx = f_runge(x)
    dfdx = -32*x*fx / (1+(4.*x)**2)
    return -(64*x*dfdx + 32*fx) / (1+(4.*x)**2)

def f_step(x): return x > 0.             ### Heaviside fn
def d2fdx2_step(x): return x * 0.

def f_kink(x): return np.abs(x)          ### Kink fn
def d2fdx2_kink(x): return x * 0.

f = f_runge
d2fdx2 = d2fdx2_runge

In [ ]:
fi = f(xi)                         ### Sample values 

### Plotting
xx = np.linspace(Ia,Ib,101)
plt.plot(xx, f(xx))
plt.plot(xi, f(xi), 'ok')
plt.plot(xx, d2fdx2(xx))

plt.xlabel(r'$x$'); plt.ylabel(r'$f$')

### Cubic-spline

Instead of interpolating all the samples with a single, high-degree polynomial, we can also use a collection of low-degree polynomials on <i>sub-intervals</i> of $[a,b]$. The original interval is divided into non-overlapping sub-intervals and a different polynomial fit can be used on each sub-interval. A linear polynomial on each interval would make the interpolation globally continuous, but non-differentiable at the samples.  By using higher-order polynomials we can enforce continuous derivatives as extra conditions. These collections of polynomials are called 'splines' (see notes, pg. 40).

**Exercise 1:
(a)  Speculate on some advanges and disadvantages of the above strategy.**

One of the most popular choices is to use cubic polynomials, i.e. <i>cubic splines</i>:
$$
S_j(x) := a_j + b_j (x-x_j) + c_j (x-x_j)^2 + d_j (x-x_j)^3 \quad\mbox{for}\quad x\in[x_i,x_{i+1}] \quad\mbox{for}\quad j\in\{0,\dots, N-1\}
$$
where $j$ indicates the interval. We have 4 coefficients on each interval, and for convenience $x$ has been offset by the start of each interval $(x - x_j)$.

This choice of $S_j(x)$ has the very nice property that for $x = x_j$:
$$
\begin{align}
S_j(x_j) &= a_j \\
S'_j(x_j) &= b_j \\
S''_j(x_j) &= 2c_j \\
S'''_j(x_j) &= 6d_j
\end{align}
$$
so that only one coefficient is involved in the derivative of any order.  This will be useful in the boundary-conditions later.

In [ ]:
def S(x, xj, a, b, c, d):
    return a + b * (x-xj) + c * (x-xj)**2 + d * (x-xj)**3

**(b) What is the maximum number of roots of a cubic?  The maximum number of minima and maxima?  Will it be possible for the spline to oscillate as did higher-order polynomials?**

Where now the coefficients $a$ to $d$ are defined <i>per sub-interval</i>. For splines we require, not only that the spline passes through $(x_i, f(x_i))$ (<b>interpolation</b>), but also that the 1st and 2nd derivatives are continuous at the interior nodes (<b>smoothness</b>):

$$
\begin{align}
S_j(x_{j+1}) &= S_{j+1}(x_{j+1}) = f(x_{j+1})&\quad\mbox{for}\quad &0 \leq j \leq N-2\\
S'_j(x_{j+1}) &= S'_{j+1}(x_{j+1}) &\quad\mbox{for}\quad &0 \leq j \leq N-2 \\
S''_j(x_{j+1}) &= S''_{j+1}(x_{j+1})&\quad\mbox{for}\quad &0 \leq j \leq N-2
\end{align}
$$

At the boundaries, the interpolation conditions become:
$$
\begin{align}
S_0(x_0) &= f(x_0) \\
S_{N-1}(x_N) &= f(x_N)
\end{align}
$$

This gives a total of $4(N-1) + 2$ conditions for $4N$ unknown coefficients.  So we need two more <i>boundary-conditions (BCs)</i>. The way we define these boundary conditions will define different types of cubic splines.

**(c) Consider the case of enforcing smoothness of <i>third</i> derivatives.  How many unknowns are required to satisfy all conditions?  Would quartic polynomials on each interval be sufficient?  How manys BCs would be required?**

#### Natural Boundary Conditions and Known BCs

To get so-called "natural" BCs, we set 2nd-derivatives of the spline to $0$ at the boundary.  In particular to set the 2nd-derivative at the boundary to $0$, we only need to set $c_0 = c_N = 0$.  On the other hand, if the 2nd-derivative at the boundary is known $f''(x_0) = F$, then this can be enforced by setting $c_0 = \frac{1}{2}F$.

#### Solving for coefficients

In the lecture notes we see how these conditions can be transformed into a linear system of equations for $c_i$, from which $a$, $b$ and $d$ can be explicitly calculated.  Here $h_j$ is the width of interval $j$:

In [ ]:
h = xi[1:] - xi[:-1]       # width of interval i - vector size N

The linear system for the $c$-coefficients can be expressed in matrix form and has left-hand side (LHS):

In [ ]:
def system_matrix(grid):
    N = len(grid)-1
    h = grid[1:] - grid[:-1]   # width of interval i - vector size N
    A = np.zeros((N+1,N+1))
    ### Equations describing conditions at interior nodes
    for i in range(1, N):
        A[i,i]   = 2*(h[i-1]+h[i])
        A[i,i-1] = h[i-1]
        A[i,i+1] = h[i]
    return A
A = system_matrix(xi)

#### (e) Implement the natural spline boundary conditions in A.

In [ ]:
#A[?,?] = ?   ### TODO - boundary conditions

#### (f) The matrix has a simple tri-diagonal structure - the vast majortity of the entries are zero, and this makes it much easier to solve for large $N$ than the Vandermonde matrix. Verify this by typing plt.imshow(A, interpolation='none').

The right-hand side (RHS) of the system for $c$ can be viewed as an approximation of 2nd-derivatives of $f(x)$ (we will see this in the next module on numerical differentiation).  This makes sense since $c$ control the 2nd-derivatives of the spline at the nodes (see equation of cucbic spline above):

In [ ]:
def rhs(grid, fi):           ### RHS - interior points
    N = len(grid)-1
    h = grid[1:] - grid[:-1]   # width of interval i - vector size N
    rhs = np.zeros(N+1)
    for i in range(1, N):
        rhs[i] = 3.*(fi[i+1] - fi[i])/h[i] - 3.*(fi[i] - fi[i-1])/h[i-1]
    return rhs
frhs = rhs(xi, fi)

In [ ]:
#### Natural boundary conditions
#frhs[0] = ???    ###TODO
#frhs[-1] = ???   ###TODO

There are better methods to solve tridiagonal systems (an efficient form of Gaussian elimination is possible), but here we just use brute force since $N$ is small enough:

In [ ]:
# Solve system
c = np.linalg.solve(A, frhs)

In [ ]:
# Compute remaining coeffs
a = fi[:]                                                  # N+1
b = (a[1:] - a[:-1]) / h[:] - h[:]/3. * (2*c[:-1] + c[1:]) # N
d = (c[1:] - c[:-1]) / (3. * h[:])                         # N

print('a = ',a)
print('b = ',b)
print('c = ',c)
print('d = ',d)

### Plotting the spline

**Exercise 2:**

**(a) What happens to splines when the number of samples $N$ increses? How is their stability? [Use the function below, which performs spline interpolation, starting from samples, to function reconstruction.]**

In [ ]:
def spline_natural(xi, fi, xx):
    """
    One-shot function for spline interpolation (with natural BCs).
    
    Args:
      xi (array, n+1): Sample locations
      fi (array, n+1): Sample values
      xx (array, M):   Reconstuction locations
    Return:
      ff (array, M): Reconstructed values at xx
    """
    h = xi[1:] - xi[:-1]       # Interval width
    N = len(h)
                               ### Setup system
    A = system_matrix(xi)      # Left-hand side 
    frhs = rhs(xi, fi)         # Right-hand side
    A[0,0] = A[N,N] = 1        # BC for LHS (natural)
    frhs[0] = 0                # BC for RHS (natural)
    frhs[-1] = 0 
                               ### Solve system for coefficients
    c = np.linalg.solve(A, frhs)
    a = fi[:]                                                  # N+1
    b = (a[1:] - a[:-1]) / h[:] - h[:]/3. * (2*c[:-1] + c[1:]) # N
    d = (c[1:] - c[:-1]) / (3. * h[:])                         # N
                               ### Reconstuct spline at locations xx  
    ii = np.digitize(xx, xi)   # Find to which interval each xx belongs
    ii = np.fmin(np.fmax(ii-1,0),N-1) 
    ff = np.zeros(xx.shape)
    for j, i in enumerate(ii): # Compute spline for each x
        ff[j] = S(xx[j], xi[i], a[i], b[i], c[i], d[i])
    return ff

In [ ]:
#### TODO - Question (a) - Change N and see how splines behave.
N = 4                              ### Number of sub-intervals
xi = np.linspace(Ia,Ib,N+1)        ### Uniform sample locs  
fi = f(xi)                         ### Sample values 

spline_natural(xi, fi, xx)

**(b) How well do splines perform with randomly located samples? Compare this also with polynomial interpolation. [Hint: Generate $N-1$ random numbers in $[0,1]$ with np.random.random(N-1) - and remember to sort them!]**

In [ ]:
#### TODO - Question (b) - Substitute uniform samples with randomly located samples

**(c)  Modify f(x) to be a function with discontinuities, or discontinuous 1st- or 2nd-derivatives. How does the spline compare to polynomial approximation in these cases? Plot the spline, the exact function and the polynomial.**

In [ ]:
#### TODO - Question (c) - See how splines behave with discontinuous functions.

### Error analysis

For a spline $s(x)$ define the error
$$
\epsilon_N := \| f - s \|_\infty := \max_{x\in[a,b]} |  f(x) - s(x) |,
$$
i.e. the maximum distance between the spline and the exact function. From theory a spline with natural BCs should have an error which converges like $\epsilon_N \sim \mathcal{O}(\frac{1}{N^2})$, for $f\in C^4([a,b])$. Again from theory: if the 2nd-derivatives of $f$ at the endpoints are known, and these are used as BCs instead of the natural BCs, then the error behaves like $\epsilon_N \sim \mathcal{O}(\frac{1}{N^4})$.

**Exercise 3:**

**(a) Verify, with a sufficiently smooth test function $f$, that the error of a natural spline converges like $\epsilon_N \sim \mathcal{O}(\frac{1}{N^2})$.  [Note: You'll need to implement an approximation of $\epsilon_N$, see for example the notebook for Module 2.**

In [ ]:
#### TODO (exercsie 3)

**(b)  Replace the natural spline boundary condition with the exact value of the second derivative at the boundaies. How does the error behaves now? What is the advantage of this method? [Hint: you have to compute and implement the second derivative of the function yourself. Evaluate it at the boundaries and substitute the appropriate values in A and RHS.]**